# Data Wrangling - Top 1000 Harry Potter (HP) Fics on AO3

In [76]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime

In [77]:
# Load data
part_1 = pd.read_csv('harry_potter_fics_by_kudos_pg1_to_10.csv')
part_2 = pd.read_csv('harry_potter_fics_by_kudos_pg11_to_20.csv')
part_3 = pd.read_csv('harry_potter_fics_by_kudos_pg21_to_30.csv')
part_4 = pd.read_csv('harry_potter_fics_by_kudos_pg31_to_40.csv')
part_5 = pd.read_csv('harry_potter_fics_by_kudos_pg41_to_50.csv')

top_1000_hp_fics_raw = pd.concat([part_1, part_2, part_3, part_4, part_5], ignore_index=True)
print(top_1000_hp_fics_raw.head()) 
print(top_1000_hp_fics_raw.tail()) # Preview data

         work_id       user_id                                 title  \
0  work_10057010  user-2878526                   All the Young Dudes   
1    work_234222     user-6004  Then Comes a Mist and a Weeping Rain   
2  work_14454174  user-3861138                              Manacled   
3   work_3101099   user-276513            The Standard You Walk Past   
4  work_12006417    user-53377                  survival is a talent   

                 username                           fandoms  \
0            MsKingBean89  ['Harry Potter - J. K. Rowling']   
1  Faith Wood (faithwood)  ['Harry Potter - J. K. Rowling']   
2                senlinyu  ['Harry Potter - J. K. Rowling']   
3            bafflinghaze  ['Harry Potter - J. K. Rowling']   
4        ShanaStoryteller  ['Harry Potter - J. K. Rowling']   

                  rating                                           warnings  \
0                 Mature  ['No Archive Warnings Apply', 'Major Character...   
1               Explicit      

In [78]:
# Check to make sure no works are accidentally repeated
print(top_1000_hp_fics_raw.work_id.value_counts())
# No work appears more than once, so we are in the clear! 

work_10057010    1
work_688863      1
work_10057286    1
work_2469818     1
work_20362597    1
                ..
work_520485      1
work_14752685    1
work_34766041    1
work_4762385     1
work_24155128    1
Name: work_id, Length: 1000, dtype: int64


In [79]:
# Let's investigate the data a bit.
print(top_1000_hp_fics_raw.describe())


              words      comments          kudos     bookmarks          hits
count  1.000000e+03   1000.000000    1000.000000   1000.000000  1.000000e+03
mean   9.811446e+04   1349.854000    9719.121000   2338.673000  2.192767e+05
std    1.433262e+05   2039.711351    6868.129422   1901.743969  3.406467e+05
min    3.660000e+02     39.000000    5347.000000    202.000000  3.007300e+04
25%    1.241950e+04    259.750000    6214.750000   1297.500000  9.378125e+04
50%    4.409450e+04    651.500000    7473.000000   1871.500000  1.429840e+05
75%    1.207150e+05   1584.250000   10329.250000   2661.000000  2.401215e+05
max    1.333997e+06  27201.000000  124810.000000  25406.000000  8.502144e+06


In [80]:
print(top_1000_hp_fics_raw.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   work_id           1000 non-null   object
 1   user_id           1000 non-null   object
 2   title             1000 non-null   object
 3   username          1000 non-null   object
 4   fandoms           1000 non-null   object
 5   rating            1000 non-null   object
 6   warnings          1000 non-null   object
 7   slash_categories  1000 non-null   object
 8   status            1000 non-null   object
 9   date_updated      1000 non-null   object
 10  relationships     1000 non-null   object
 11  characters        1000 non-null   object
 12  freeforms         1000 non-null   object
 13  summary           1000 non-null   object
 14  language          1000 non-null   object
 15  words             1000 non-null   int64 
 16  chapters          1000 non-null   object
 17  comments       

All of the data has been successfully loaded and combined into one larger dataframe. 

## Data Cleaning
While the numerical data is fine as is, a lot of the categorical data needs to be stripped and cleaned up. The data that appears as lists will be transfered to individual boolean data frames with the work_id as the primary key.

In [81]:
# Let's take a look at what our columns are.
print(top_1000_hp_fics_raw.columns)

Index(['work_id', 'user_id', 'title', 'username', 'fandoms', 'rating',
       'warnings', 'slash_categories', 'status', 'date_updated',
       'relationships', 'characters', 'freeforms', 'summary', 'language',
       'words', 'chapters', 'comments', 'kudos', 'bookmarks', 'hits'],
      dtype='object')


In [82]:
# Let's specify our data types as needed
top_1000_hp_fics_raw = top_1000_hp_fics_raw.astype({
    'work_id': 'string', 
    'user_id': 'string', 
    'title': 'string', 
    'username': 'string',
    'fandoms': 'string',
    'rating': 'string',
    'warnings': 'string',
    'slash_categories': 'string',
    'freeforms': 'string',
    'status': 'string', 
    'date_updated': 'string', 
    'relationships': 'string',
    'characters': 'string',
    'summary': 'string', 
    'language': 'string', 
    'chapters': 'string'})

# The ratings column should be categorical based on the increasing restrictions of the ratings
top_1000_hp_fics_raw['rating'] = pd.Categorical(top_1000_hp_fics_raw['rating'], ordered=True, categories=['General Audiences', 'Teen And Up Audiences', 'Mature', 'Explicit', 'Not Rated']) 

# Some of our columns are a currently a list, recognized as a string. 
# We need to clean this up so that we can use aggregate functions on the data.
top_1000_hp_fics_raw['fandoms'] = top_1000_hp_fics_raw.fandoms.apply(eval)
top_1000_hp_fics_raw['freeforms'] = top_1000_hp_fics_raw.freeforms.apply(eval)
top_1000_hp_fics_raw['warnings'] = top_1000_hp_fics_raw.warnings.apply(eval)
top_1000_hp_fics_raw['slash_categories'] = top_1000_hp_fics_raw.slash_categories.apply(eval)
top_1000_hp_fics_raw['relationships'] = top_1000_hp_fics_raw.relationships.apply(eval)
top_1000_hp_fics_raw['characters'] = top_1000_hp_fics_raw.characters.apply(eval)

In [83]:
# The following columns should be fine now. Let's take a look to make sure no adjustments need to be made:
print(top_1000_hp_fics_raw.work_id.head())
print(top_1000_hp_fics_raw.user_id.head())
print(top_1000_hp_fics_raw.title.head())
print(top_1000_hp_fics_raw.username.head())
print(top_1000_hp_fics_raw.rating.head())
print(top_1000_hp_fics_raw.summary.head()) 
print(top_1000_hp_fics_raw.language.head())
print(top_1000_hp_fics_raw.words.head())
print(top_1000_hp_fics_raw.comments.head())
print(top_1000_hp_fics_raw.kudos.head())
print(top_1000_hp_fics_raw.bookmarks.head())
print(top_1000_hp_fics_raw.hits.head())

0    work_10057010
1      work_234222
2    work_14454174
3     work_3101099
4    work_12006417
Name: work_id, dtype: string
0    user-2878526
1       user-6004
2    user-3861138
3     user-276513
4      user-53377
Name: user_id, dtype: string
0                     All the Young Dudes
1    Then Comes a Mist and a Weeping Rain
2                                Manacled
3              The Standard You Walk Past
4                    survival is a talent
Name: title, dtype: string
0              MsKingBean89
1    Faith Wood (faithwood)
2                  senlinyu
3              bafflinghaze
4          ShanaStoryteller
Name: username, dtype: string
0                   Mature
1                 Explicit
2                 Explicit
3                   Mature
4    Teen And Up Audiences
Name: rating, dtype: category
Categories (5, object): ['General Audiences' < 'Teen And Up Audiences' < 'Mature' < 'Explicit' < 'Not Rated']
0    LONG fic charting the marauders' time at Hogwa...
1    It always rains

Originally, all of the data looked great with the exception of the summary data. Upon further inspection, I noticed that the xpath for the summary data in the webscraper was incorrect. I have since resolved this problem, which hugely sped up the data collection. The data here is the second attempt to collect this data and includes all the summaries as well. 

### Cleaning the non-listed data

In [84]:
# Creating a boolean column based on status
status_map = {'Complete Work': True, 'Work in Progress': False}
top_1000_hp_fics_raw['completed'] = top_1000_hp_fics_raw['status'].map(status_map)

In [85]:
# Separating out the chapters column into two new columns for ease of analysis
top_1000_hp_fics_raw['chapters_written'] = top_1000_hp_fics_raw['chapters'].apply(lambda x: x.split('/')[0])
top_1000_hp_fics_raw['chapters_total'] = top_1000_hp_fics_raw['chapters'].apply(lambda x: x.split('/')[1])
# Replacing the ? unknown values with null values
top_1000_hp_fics_raw['chapters_total'] = top_1000_hp_fics_raw.chapters_total.replace('?', np.NaN)

# Changing the datatype of the chapters columns so aggregate functions can be used. (Cannot use int64 due to NaN)
top_100_kudos = top_1000_hp_fics_raw.astype({
    'chapters_written': 'int64', 
    'chapters_total': 'float64'})

In [132]:
# Turning date updated into a datetime date
top_1000_hp_fics_raw['datetime_updated'] = top_1000_hp_fics_raw['date_updated'].apply(lambda x: datetime.strptime(x, '%d %b %Y'))
print(top_1000_hp_fics_raw.datetime_updated.dtype) #Success!


datetime64[ns]


### Cleaning the listed data

In [133]:
#Create a list of unique items for every column. These will become the columns of our boolean dataframes.
#Fandoms
unique_fandoms = []
for row in top_1000_hp_fics_raw['fandoms']:
    for item in row:
        if item in unique_fandoms:
            None
        else:
            unique_fandoms.append(item)

#Freeforms
unique_freeforms = []
for row in top_1000_hp_fics_raw['freeforms']:
    for item in row:
        if item in unique_freeforms:
            None
        else:
            unique_freeforms.append(item)

#Warnings
unique_warnings = []
for row in top_1000_hp_fics_raw['warnings']:
    for item in row:
        if item in unique_warnings:
            None
        else:
            unique_warnings.append(item)

#Slash Categories
unique_slash_categories = []
for row in top_1000_hp_fics_raw['slash_categories']:
    for item in row:
        if item in unique_slash_categories:
            None
        else:
            unique_slash_categories.append(item)

#Relationships
unique_relationships = []
for row in top_1000_hp_fics_raw['relationships']:
    for item in row:
        if item in unique_relationships:
            None
        else:
            unique_relationships.append(item)

#Characters
unique_characters = []
for row in top_1000_hp_fics_raw['characters']:
    for item in row:
        if item in unique_characters:
            None
        else:
            unique_characters.append(item)


In [134]:
# Function to create a boolean dataframe for each listed column
def boolean_df(col_name, unique_items):
# Create empty dict to hold all the data
    bool_dict = {} 
    for item in unique_items:
        # Create an empty dict to hold the data unique to that item
        item_dict = {}
        #Iterating through 100 rows of data
        for i in range(0,100):
            work_id = top_1000_hp_fics_raw['work_id'].iloc[i]
            if item in top_1000_hp_fics_raw[col_name].iloc[i]:
                # True if that item is in the list for that work
                item_dict[work_id] = True
            else:
                item_dict[work_id] = False
        # Add boolean dictionary for specific item to larger dict
        bool_dict[item] = item_dict
    return pd.DataFrame(bool_dict)


In [135]:
# Create a separate dataframe for each listed column
hp_fandoms_bool = boolean_df('fandoms', unique_fandoms)
hp_freeforms_bool = boolean_df('freeforms', unique_freeforms)
hp_warnings_bool = boolean_df('warnings', unique_warnings)
hp_slash_categories_bool = boolean_df('slash_categories', unique_slash_categories)
hp_relationships_bool = boolean_df('relationships', unique_relationships)
hp_characters_bool = boolean_df('characters', unique_characters)

In [136]:
# Let's check these to see how they turned out.
print(hp_fandoms_bool.head())
print(hp_freeforms_bool.head())
print(hp_warnings_bool.head())
print(hp_slash_categories_bool.head())
print(hp_relationships_bool.head())
print(hp_characters_bool.head())

               Harry Potter - J. K. Rowling  The Avengers (Marvel Movies)  \
work_10057010                          True                         False   
work_234222                            True                         False   
work_14454174                          True                         False   
work_3101099                           True                         False   
work_12006417                          True                         False   

               Doctor Who  Sherlock (TV)  \
work_10057010       False          False   
work_234222         False          False   
work_14454174       False          False   
work_3101099        False          False   
work_12006417       False          False   

               Sherlock Holmes - Arthur Conan Doyle  \
work_10057010                                 False   
work_234222                                   False   
work_14454174                                 False   
work_3101099                                  False 

### Trimming the fat
Now that the data is clean or has been transferred to boolean dataframe, it's time to remove any of the columns that are no longer necessary.

In [139]:
# Drop the columns that are redundant (like date_updated and freeforms) or not going to be used by myself (like username)
top_1000_hp_fics_clean = top_1000_hp_fics_raw.drop([
    'username', 
    'fandoms', 
    'warnings', 
    'slash_categories', 
    'relationships', 
    'characters', 
    'freeforms', 
    'status', 
    'chapters', 
    'date_updated'
    ], axis=1)

print(top_1000_hp_fics_clean.head()) #Looks great!

         work_id       user_id                                 title  \
0  work_10057010  user-2878526                   All the Young Dudes   
1    work_234222     user-6004  Then Comes a Mist and a Weeping Rain   
2  work_14454174  user-3861138                              Manacled   
3   work_3101099   user-276513            The Standard You Walk Past   
4  work_12006417    user-53377                  survival is a talent   

                  rating                                            summary  \
0                 Mature  LONG fic charting the marauders' time at Hogwa...   
1               Explicit  It always rains for Draco Malfoy. Metaphorical...   
2               Explicit  Harry Potter is dead. In the aftermath of the ...   
3                 Mature  On returning to Hogwarts for their Eighth Year...   
4  Teen And Up Audiences  In the middle of their second year, Draco and ...   

  language   words  comments   kudos  bookmarks     hits  completed  \
0  English  526969   

## Saving the work!
All the cleaned data can now be saved to new csv files to be used in future analysis.

In [140]:
top_1000_hp_fics_clean.to_csv('top_1000_hp_fics_clean.csv')
hp_characters_bool.to_csv('top_1000_hp_characters_bool.csv')
hp_fandoms_bool.to_csv('top_1000_hp_fandoms_bool.csv')
hp_freeforms_bool.to_csv('top_1000_hp_freeforms_bool.csv')
hp_relationships_bool.to_csv('top_1000_hp_relationships_bool.csv')
hp_slash_categories_bool.to_csv('top_1000_hp_slash_categories_bool.csv')
hp_warnings_bool.to_csv('top_1000_hp_warnings_bool.csv')